In [1]:
!pip3 install dcor
!pip3 install networkx
!pip3 install PyPortfolioOpt
!pip3 install alpaca-trade-api
!pip3 install trading-calendars


import alpaca_trade_api as tradeapi
import requests, json

import dcor
import networkx as nx
from pypfopt import discrete_allocation
from pypfopt.expected_returns import mean_historical_return
from pypfopt.efficient_frontier import EfficientFrontier

import sklearn
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import pytz
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

     |████████████████████████████████| 3.2MB 10.6MB/s 
     |████████████████████████████████| 25.3MB 1.3MB/s 
  Found existing installation: llvmlite 0.31.0
    Uninstalling llvmlite-0.31.0:
      Successfully uninstalled llvmlite-0.31.0
  Found existing installation: numba 0.48.0
    Uninstalling numba-0.48.0:
      Successfully uninstalled numba-0.48.0
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 204kB 23.9MB/s 
     |████████████████████████████████| 112kB 23.5MB/s 
  Created wheel for trading-calendars: filename=trading_calendars-2.1.1-cp36-none-any.whl size=140920 sha256=f32a296694d8b7e47778cd14a895056170baca84f798fd2360003b01a0af9cba
  Stored in directory: /root/.cache/pip/wheels/79/92/44/de8b4d9a7d86cd8f67ea3adfa91bdc7bd441c691b733418cca
Successfully built trading-calendars


/usr/local/lib/python3.6/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


Hyperparameters

In [16]:
#Put your target companies in "target_list"
target_list = ['AMZN', 'MA', 'GOOG', 'ADBE', 'MSFT', 'AAPL', 'V', 'DIS', 'DPZ', 'PYPL', 'FB', 'COST']

#Start date for 
start_date = '2018-01-01'

APCA_API_KEY_ID = 'PKCHJEA5FAGJ76MD8QNK'
APCA_API_SECRET_KEY = 'KN9dNepezJveDSZEho8bWmGAvwXXPxM5CLxTUCsf'
BASE_URL = 'https://paper-api.alpaca.markets'

now = datetime.now(pytz.timezone('US/Eastern'))
lookback = 60

training_pct = 1.0

In [2]:
#############################################################################
def df_distance_correlation(df_train):
    df_train_dcor = pd.DataFrame(index=stocks, columns=stocks)
    k=0

    for i in stocks:
        v_i = df_train.loc[:, i].values
        for j in stocks[k:]:
            v_j = df_train.loc[:, j].values
            dcor_val = dcor.distance_correlation(v_i, v_j)
            df_train_dcor.at[i,j] = dcor_val
            df_train_dcor.at[j,i] = dcor_val
        k+=1

    return df_train_dcor
#############################################################################
def build_corr_nx(df_train):
    cor_matrix = df_train.values.astype('float')
    sim_matrix = 1 - cor_matrix
    
    G = nx.from_numpy_matrix(sim_matrix)
    stock_names = df_train.index.values
    G = nx.relabel_nodes(G, lambda x: stock_names[x])

    G.edges(data=True)
    H = G.copy()
    
    for (u, v, wt) in G.edges.data('weight'):
        if wt >= 1 - 0.325:
#hyperparameter
            H.remove_edge(u, v)
        if u == v:
            H.remove_edge(u, v)
     
    return H
#############################################################################
def centrality_to_portfolio_weights(weights):
    for key, value in weights.items():
        weights[key] = 1/value
    norm = 1.0 / sum(weights.values())

    for key in weights:
        weights[key] = round(weights[key] * norm, 3)
    return weights
#############################################################################
def get_account():
    r = requests.get(ACCOUNT_URL, headers=HEADERS)

    return json.loads(r.content)
#############################################################################
def create_order(symbol, qty, side, type, time_in_force):
    data = {
        "symbol": symbol,
        "qty": qty,
        "side": side,
        "type": type,
        "time_in_force": time_in_force
    }

    r = requests.post(ORDERS_URL, json=data, headers=HEADERS)

    return json.loads(r.content)
#############################################################################
def get_orders():
    r = requests.get(ORDERS_URL, headers=HEADERS)

    return json.loads(r.content)
#############################################################################
ACCOUNT_URL = "{}/v2/account".format(BASE_URL)
ORDERS_URL = "{}/v2/orders".format(BASE_URL)
HEADERS = {'APCA-API-KEY-ID': APCA_API_KEY_ID, 'APCA-API-SECRET-KEY': APCA_API_SECRET_KEY}
api = tradeapi.REST(APCA_API_KEY_ID, APCA_API_SECRET_KEY, base_url=BASE_URL)
account = api.get_account()
today = date.today()
clock = api.get_clock()
#############################################################################

In [17]:
stuff = []
for i in target_list:
  globals()[i] =  api.polygon.historic_agg_v2(i , 1, 'day', _from=start_date, to=today).df
  globals()[i]['Ticker'] = i
  stuff.append(globals()[i])

empty = stuff[0].iloc[:0,:]
for i in range(0, len(stuff)):
  empty = empty.append(stuff[i])


empty_close = empty.pivot(columns='Ticker', values='close')
empty_close_copy = empty_close.copy()

data_training_raw, data_test_raw = np.split(empty_close, [int(training_pct*len(empty_close))])
array_data_training = empty_close.values

date_index = pd.DataFrame(data_training_raw.index)
for i in range(0, len(date_index)):
  date_index.iloc[i,0] = date_index.iloc[i,0].date()

date_index = date_index[lookback:]
date_index = date_index.reset_index().iloc[:,1:]
date_index.loc[len(date_index)] = clock.next_open.date()

In [18]:
historical_prices_plus_one = date_index

for c in range(0, len(target_list)):
      X_train = []
      y_train = []

      print(target_list[c])

      for i in range(lookback, array_data_training.shape[0]):
          X_train.append(array_data_training[i-lookback:i])
          y_train.append(array_data_training[i, c])

      X_train, y_train = np.asarray(X_train).astype('float32'), np.asarray(y_train).astype('float32')
      X_train, y_train = np.array(X_train), np.array(y_train)
 

      model = Sequential()
      model.add(LSTM(units = 60, activation = 'tanh', return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
      model.add(Dropout(0.5))
      model.add(LSTM(units = 60, activation = 'tanh', return_sequences = True))
      model.add(Dropout(0.5))
      model.add(LSTM(units = 80, activation = 'tanh', return_sequences = True))
      model.add(Dropout(0.5))
      model.add(LSTM(units = 120, activation = 'tanh'))
      model.add(Dropout(0.5))
      model.add(Dense(units = 1))

      model.compile(optimizer='sgd', loss = 'mean_squared_error')
      model.fit(X_train, y_train, epochs=1, batch_size=64, verbose=0)

      inputs = np.array(data_training_raw.tail(lookback))
      inputs_reshaped = inputs.reshape(1, inputs.shape[0], inputs.shape[1])

      y_pred = model.predict(inputs_reshaped)
      
      new_shit = np.append(y_train, y_pred)
      new_shit_df = pd.DataFrame(data=new_shit)
      historical_prices_plus_one[target_list[c]] = new_shit_df

df_train_close = historical_prices_plus_one.set_index("timestamp")
df_train_close.index = pd.to_datetime(df_train_close.index)

AMZN
MA
GOOG
ADBE
MSFT
AAPL
V
DIS
DPZ
PYPL
FB
COST


In [19]:
df_train_close_copy = df_train_close.copy()
stocks = df_train_close.columns.tolist()

df_train_list = [df_train_close]

for df in df_train_list:
    for s in stocks:
        df[s] = df[s].diff()
#redundant if you normalize from the get go @ LSTM input stage by taking pct change

for df in df_train_list:
    df.dropna(inplace=True)

df_buy_in = df_train_close_copy.loc[today.strftime("%Y-%m-%d")].sort_index().to_frame('Buy In')

In [20]:
df_train_dcor_list = [df_distance_correlation(df) for df in df_train_list]
H_close = build_corr_nx(df_train_dcor_list[0])
H_master = H_close

In [21]:
weights = nx.communicability_betweenness_centrality(H_master)
centrality_to_portfolio_weights(weights)

{'AAPL': 0.43,
 'ADBE': 0.053,
 'AMZN': 0.053,
 'COST': 0.053,
 'DIS': 0.053,
 'DPZ': 0.053,
 'FB': 0.053,
 'GOOG': 0.049,
 'MA': 0.049,
 'MSFT': 0.053,
 'PYPL': 0.049,
 'V': 0.053}

In [47]:
capital = float(api.get_account().portfolio_value)

alloc = discrete_allocation.DiscreteAllocation(
    weights, 
    df_buy_in['Buy In'], 
    total_portfolio_value=capital
)

alloc = alloc.greedy_portfolio()[0]


alloc_series = pd.Series(alloc, name='Desired Shares')
alloc_series.index.name = 'Assets'
alloc_series.reset_index
df_alloc = alloc_series.sort_index().to_frame('Shares')

In [48]:
test = pd.DataFrame(df_alloc.values*df_buy_in.values,columns=["Total Price"], index=df_alloc.index)

allocation = pd.concat([df_alloc,df_buy_in,test],axis=1)
print(allocation)

print("\n")

counter = 0
for i in range(0, len(test[0:])):
  counter += test.iloc[i,0]
  print(counter)

        Shares       Buy In   Total Price
Assets                                   
AAPL       115   372.649994  42854.749298
ADBE        15   354.470001   5317.050018
AMZN        41   127.830002   5241.030075
COST        26   201.660004   5243.160095
DIS          2  1790.859985   3581.719971
DPZ         16   325.940002   5215.040039
FB          22   247.250000   5439.500000
GOOG         2  3120.760010   6241.520020
MA          11   456.500000   5021.500000
MSFT        31   172.259995   5340.059830
PYPL        23   216.440002   4978.120056
V           20   261.100006   5222.000122


42854.7492980957
48171.79931640625
53412.82939147949
58655.989486694336
62237.70945739746
67452.74949645996
72892.24949645996
79133.76951599121
84155.26951599121
89495.32934570312
94473.44940185547
99695.44952392578


In [31]:
portfolio = api.list_positions()

df_current = pd.DataFrame(columns = ['Assets', 'Current Shares']) 

for i in range(len(portfolio)):
  #portfolio[i].side
  print(portfolio[i].symbol, portfolio[i].qty)
  df_current = df_current.append({'Assets':portfolio[i].symbol,'Current Shares':float(portfolio[i].qty)}, ignore_index=True)
df_current = df_current.set_index("Assets")
df_current

MSFT 1
AMZN 1
AAPL 1


,Current Shares
Assets,
MSFT,1.0
AMZN,1.0
AAPL,1.0


In [32]:
reconcil_df = pd.concat([df_alloc, df_current], axis=1).fillna(0)
reconcil_df['Qty Diff'] = reconcil_df['Shares'] - reconcil_df['Current Shares']
reconcil_df = reconcil_df.sort_values(by="Qty Diff")

In [35]:
for i in range(0, len(reconcil_df)):
  symbol = reconcil_df['Qty Diff'].index[i]
  qty = abs(reconcil_df['Qty Diff'][i])
  if reconcil_df['Qty Diff'][i]>0:
    side = 'buy'
  else:
    side = 'sell'
  type = 'market'
  time_in_force = 'gtc'
  #create_order(symbol, qty, side, type, time_in_force)
  print(symbol, qty, side, type, time_in_force)


DIS 2.0 buy market gtc
GOOG 2.0 buy market gtc
MA 11.0 buy market gtc
ADBE 15.0 buy market gtc
DPZ 16.0 buy market gtc
V 20.0 buy market gtc
FB 22.0 buy market gtc
PYPL 23.0 buy market gtc
COST 26.0 buy market gtc
MSFT 30.0 buy market gtc
AMZN 40.0 buy market gtc
AAPL 114.0 buy market gtc
